In [ ]:
#default_exp visualizations

# Visualizations

> Functions for showing detection or segmentation results

In [ ]:
# export

from icevision.all import *
from fastcore.dispatch import *
from fastai.data.all import *
from fastai.vision.all import *
from matplotlib.cm import ScalarMappable


## Custom show functions for MultiChannelTensorImage

In [ ]:
# export

def show_composite(img, channels, ax=None, figsize=(3,3), title=None, scale=True,
                   ctx=None, norm_min=None, norm_max=None, scale_axis=(0,1), **kwargs)->plt.Axes:
    "Show three channel composite so that channels correspond to R, G and B"
    ax = ifnone(ax, ctx)
    if ax is None: _, ax = plt.subplots(figsize=figsize)    
    r, g, b = channels
    tempim = img.data.cpu().numpy()
    im = np.zeros((tempim.shape[1], tempim.shape[2], 3))
    im[...,0] = tempim[r]
    im[...,1] = tempim[g]
    im[...,2] = tempim[b]

    if scale: im = norm(im, norm_min, norm_max, scale_axis)
    ax.imshow(im, **kwargs)
    ax.axis('off')
    if title is not None: ax.set_title(title)
    return ax

def show_single_channel(img, channel, ax=None, figsize=(3,3), ctx=None, 
                        title=None, **kwargs) -> plt.Axes:
    ax = ifnone(ax, ctx)
    if ax is None: _, ax = plt.subplots(figsize=figsize)    
    tempim = img.data.cpu().numpy()
    ax.imshow(norm(tempim[channel], norm_min=tempim[channel].min(), norm_max=tempim[channel].max()), **kwargs)
    ax.axis('off')
    if title is not None: ax.set_title(title)
    return ax

def show_normalized_spectral_index(img, channels, ax=None, figsize=(3,3), ctx=None,
                                   title=None, **kwargs) -> plt.Axes:
    "Show normalized spectral index such as NDVI"
    ax = ifnone(ax, ctx)
    if ax is None: _, ax = plt.subplots(figsize=figsize)
    b_0, b_1 = channels
    tempim = img.data.cpu().numpy()
    im = (tempim[b_0] - tempim[b_1])/(tempim[b_0] + tempim[b_1])
    ax.imshow(im, vmin=-1, vmax=1, **kwargs)
    ax.axis('off')
    if title is not None: ax.set_title(title)
    return ax

def show_mean_spectra(img, ax=None, figsize=(3,3), ctx=None, title=None, **kwargs) -> plt.Axes:
    "Show average spectra graph"
    ax = ifnone(ax, ctx)
    if ax is None: _, ax = plt.subplots(figsize=figsize)
    tempim = img.data.cpu().numpy()
    means = np.nanmean(tempim, axis=(-2, -1))
    ax.plot(means, **kwargs)
    ax.grid(True)
    if title is not None: ax.set_title(title)
    return ax
        
def norm(vals, norm_min=None, norm_max=None, axis=(0,1)):
    """
    For visualization purposes scale image with `(vals-norm_min)/(norm_max-norm_min), 
    with norm_min and norm_max either specified or within 0.01 and 0.99 quantiles of all values
    """
    norm_min = ifnone(norm_min, np.quantile(vals, 0.01, axis=axis))
    norm_max = ifnone(norm_max, np.quantile(vals, 0.99, axis=axis))
    ret_im = (vals - norm_min)/(norm_max-norm_min)
    ret_im[ret_im < 0] = 0
    ret_im[ret_im > 1] = 1
    return ret_im

## Semantic segmentation (fastai)

In [ ]:
#export

@typedispatch
def show_batch(x:TensorImage, y:TensorMask, samples, ctxs=None, max_n=6, nrows=None, ncols=2, figsize=None, **kwargs):
    if figsize is None: figsize = (ncols*6, max_n//ncols * 3)
    if ctxs is None: ctxs = get_grid(min(len(samples*2), max_n), nrows=nrows, ncols=ncols, add_vert=1, figsize=figsize,
                                     double=False, title='Image/Target')        
        
    for i in range(2):
        ctxs[::2] = [b.show(ctx=c, **kwargs) for b,c,_ in zip(samples.itemgot(0), ctxs[::2],range(2*max_n))]
        ctxs[1::2] = [b.show(ctx=c, **kwargs) for b,c,_ in zip(samples.itemgot(i), ctxs[1::2],range(2*max_n))]

    return ctxs

In [ ]:
#export

from matplotlib.cm import ScalarMappable

@typedispatch
def show_results(x:TensorImage, y:TensorMask, samples, outs, ctxs, max_n=6,
                 nrows=None, ncols=3, figsize=None, show_cbar=False, **kwargs):
    "Patch `show_results` to show segmentation results in three columns (no-mask, ground truth, prediction)"
    
    
    if ctxs is None: ctxs = get_grid(min(len(samples)*3, max_n*3), nrows=int(min(len(samples), max_n)), 
                                     ncols=3, add_vert=1, figsize=figsize, double=False, 
                                     title='Image/Target/Prediction')
        
    for i in range(2):
        ctxs[::3] = [b.show(ctx=c, **kwargs) for b,c,_ in zip(samples.itemgot(0), ctxs[::3],range(3*max_n))]
        ctxs[1::3] = [b.show(ctx=c, **kwargs) for b,c,_ in zip(samples.itemgot(i), ctxs[1::3],range(2*max_n))]
    
    for o in [samples,outs]:
        ctxs[2::3] = [b.show(ctx=c, **kwargs) for b,c,_ in zip(o.itemgot(0),ctxs[2::3], range(2*max_n))]
    
    if isinstance(y, RegressionMask) and show_cbar:
        ctxs[1::3] = [plt.colorbar(ScalarMappable(cmap='viridis'), ax=c) for c, _ in zip(ctxs[1::3], range(2*max_n))]
        ctxs[2::3] = [plt.colorbar(ScalarMappable(cmap='viridis'), ax=c) for c, _ in zip(ctxs[2::3], range(2*max_n))]

    return ctxs

## Object detection/instance segmentation (IceVision)

> Modified icevision functions to do what we want to.

In [ ]:
#export

def plot_grid_preds_actuals_raws(
    raws, actuals, predictions, figsize=None, show=False, annotations=None, **kwargs
):
    fig, axs = plt.subplots(
        nrows=len(actuals),
        ncols=3,
        figsize=figsize or (6, 6 * len(actuals) / 2 / 0.75),
        **kwargs,
    )
    i = 0
    for im, ax in zip(zip(raws, actuals, predictions), axs.reshape(-1, 3)):
        ax[0].imshow(im[0])
        ax[0].set_title('Aerial image')
        ax[1].imshow(im[1], cmap=None)
        ax[1].set_title("Expert annotations")
        ax[2].imshow(im[2], cmap=None)
        ax[2].set_title("Predicted annotations")

        if annotations is None:
            ax[0].set_axis_off()
            ax[1].set_axis_off()
            ax[2].set_axis_off()
        else:
            ax[0].set_axis_off()
            ax[1].get_xaxis().set_ticks([])
            ax[1].set_frame_on(False)
            ax[1].get_yaxis().set_visible(False)
            ax[1].set_xlabel(annotations[i][0], ma="left")

            ax[2].get_xaxis().set_ticks([])
            ax[2].set_frame_on(False)
            ax[2].get_yaxis().set_visible(False)
            ax[2].set_xlabel(annotations[i][1], ma="left")

            i += 1

    plt.tight_layout()
    if show:
        plt.show()
    return axs

def show_raw_mask_pred(    
    samples: Union[Sequence[np.ndarray], Sequence[dict]],
    preds: Sequence[dict],
    class_map: Optional[ClassMap] = None,
    denormalize_fn: Optional[callable] = denormalize_imagenet,
    display_label: bool = True,
    display_bbox: bool = True,
    display_mask: bool = True,
    ncols: int = 1,
    figsize=None,
    show=False,
    annotations=None,
) -> None:

    if not len(samples) == len(preds):
        raise ValueError(
            f"Number of imgs ({len(samples)}) should be the same as "
            f"the number of preds ({len(preds)})"
        )

    if all(type(x) is dict for x in samples):
        raws = [
            draw_sample(
                sample=sample,
                class_map=class_map,
                display_label=False,
                display_bbox=False,
                display_mask=False,
                denormalize_fn=denormalize_fn,
            )
            for sample in samples
        ]
        actuals = [
            draw_sample(
                sample=sample,
                class_map=class_map,
                display_label=display_label,
                display_bbox=display_bbox,
                display_mask=display_mask,
                denormalize_fn=denormalize_fn,
            )
            for sample in samples
        ]

        imgs = [sample["img"] for sample in samples]
        predictions = [
            draw_pred(
                img=img,
                pred=pred,
                class_map=class_map,
                denormalize_fn=denormalize_fn,
                display_label=display_label,
                display_bbox=display_bbox,
                display_mask=display_mask,
            )
            for img, pred in zip(imgs, preds)
        ]

        plot_grid_preds_actuals_raws(
            raws, actuals, predictions, figsize=figsize, show=show, annotations=annotations
        )

    else:
        partials = [
            partial(
                show_pred,
                img=img,
                pred=pred,
                class_map=class_map,
                denormalize_fn=denormalize_fn,
                display_label=display_label,
                display_bbox=display_bbox,
                display_mask=display_mask,
                show=False,
            )
            for img, pred in zip(samples, preds)
        ]
        plot_grid(
            partials, ncols=ncols, figsize=figsize, show=show, annotations=annotations
        )